In [169]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [59]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [153]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_csv('university_towns.txt', header=None, sep='\t')
    df.columns = ['combined']
    df['combined'] = df.apply(lambda x: x['combined']+'\t' if '[edit]' in x['combined'] else '\t'+x['combined'],
                              axis=1)
    df['State'] = df['combined'].str.split("\t").str[0] \
                                .str.split('[').str[0]
    df['RegionName'] = df['combined'].str.split("\t").str[1] \
                                     .str.split(' \(').str[0]
    df = df.applymap(lambda x: np.nan if (x.isspace() or x=='') else x)
    df['State'] = df['State'].fillna(method='ffill')
    df = df[df['RegionName'].notnull()].drop(['combined'], axis=1)
   
    
    return df

df = get_list_of_university_towns()



In [185]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[df.index > '2000']
    recession = []
    for i in range (np.size(df['GDP in billions of current dollars']) - 2):
        if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) < 0 and
            (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) < 0):
            recession += [df.iloc[i].name]
    return recession[0]

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[df.index >= get_recession_start()]
    non_recession = []
    for i in range (np.size(df['GDP in billions of current dollars']) - 2):
        if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) > 0 and
            (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) > 0):
            non_recession += [df.iloc[i+2].name]
    return non_recession[0]   


get_recession_end()

'2009q4'

In [11]:
def get_recession_bottom():
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[(df.index >= get_recession_start()) &
            (df.index <= get_recession_end())]
    min_gdp = np.min(df['GDP in billions of current dollars'])
    df_m_g = df[df['GDP in billions of current dollars']==min_gdp]
    return df_m_g.iloc[0].name

get_recession_bottom()

'2009q2'

In [115]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv', header=0, sep=',')
    df['State'] = df['State'].map(lambda x: states[x])
    df = df.set_index(['State', 'RegionName'])
    df = df.loc[:,'2000-01':]
    df = df.T
    df.index = pd.to_datetime(df.index)
    df = df.resample('Q').mean()
    df.index = df.index.to_period("Q")
    df = df.T
    return df


df  = convert_housing_data_to_quarters()

In [225]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = df.columns.get_loc(get_recession_bottom())
    start_m1 = df.columns.get_loc(recession_start) - 1
    df2 = df.iloc[:,[start_m1, recession_bottom]]
    df2.columns = df2.columns.astype(str)
    df2['ratio'] = df2.apply(lambda x: x[0] / x[1] if x[1] != np.nan else np.nan, axis=1)
    #ratio_df = pd.DataFrame({'ratio':ratio})
    
    unis = get_list_of_university_towns()
    unis['ut'] = 1
    unis = unis.set_index(['State', 'RegionName'])
    rat_comb = pd.merge(df2, unis, how='left', left_index=True, right_index=True)
    
    uni_town = rat_comb[rat_comb['ut'] == 1]
    nonuni_town = rat_comb[rat_comb['ut'] != 1]
    p =      ttest_ind(nonuni_town['ratio'], 
                       uni_town['ratio'], nan_policy='omit')[1]
    if p < 0.01:
        different = True
    else:
        different = False
    if np.mean(rat_comb[rat_comb['ut'] == 1].loc[:,'ratio']) > np.mean(rat_comb[rat_comb['ut'] != 1].loc[:,'ratio']):
        better = "university town"
    else:
        better = "non-university town"
    return (different, p, better)
    #return uni_town

tt = run_ttest()

C:\Users\jwharton\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [226]:
tt

(True, 0.002724063704753125, 'non-university town')

In [214]:
tt.apply(lambda x: x[0] / x[1] if x[1] != np.nan else np.nan, axis=1)

State                 RegionName         
New York              New York               1.081789
California            Los Angeles            1.213498
Illinois              Chicago                1.082840
Pennsylvania          Philadelphia           1.016930
Arizona               Phoenix                1.220131
Nevada                Las Vegas              1.413590
California            San Diego              1.133248
Texas                 Dallas                 1.097685
California            San Jose               1.153561
Florida               Jacksonville           1.139172
California            San Francisco          1.127530
Texas                 Austin                 1.047712
Michigan              Detroit                1.141986
Ohio                  Columbus               1.034012
Tennessee             Memphis                1.110526
North Carolina        Charlotte              1.065184
Texas                 El Paso                0.992137
Massachusetts         Boston            

In [206]:
df.columns.astype(str)

Index(['State', 'RegionName'], dtype='object')

In [194]:
run_ttest()

(True, 0.002724063704753125, 'non-university town')

In [186]:
get_recession_start()

'2008q3'

In [184]:
get_recession_bottom()

'2009q2'

In [227]:
df = convert_housing_data_to_quarters()
recession_start = get_recession_start()
recession_bottom = df.columns.get_loc(get_recession_bottom())
start_m1 = df.columns.get_loc(recession_start) - 1
df2 = df.iloc[:,[start_m1, recession_bottom]]
df2.columns = df2.columns.astype(str)
df2['ratio'] = df2.apply(lambda x: x[0] / x[1] if x[1] != np.nan else np.nan, axis=1)
#ratio_df = pd.DataFrame({'ratio':ratio})
   
unis = get_list_of_university_towns()
unis['ut'] = 1
unis = unis.set_index(['State', 'RegionName'])
rat_comb = pd.merge(df2, unis, how='left', left_index=True, right_index=True)
   
uni_town = rat_comb[rat_comb['ut'] == 1]
nonuni_town = rat_comb[rat_comb['ut'] != 1]

C:\Users\jwharton\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [235]:
ttest_ind(nonuni_town['ratio'], uni_town['ratio'], nan_policy='omit')

Ttest_indResult(statistic=2.998032664179151, pvalue=0.002724063704753125)

In [232]:
nonuni_town.count()

2008Q2    9599
2009Q2    9671
ratio     9599
ut           0
dtype: int64

In [244]:
nonuni_town.shape


(10461, 4)

In [237]:
df.drop_duplicates(keep=False).shape

(10730, 67)

In [238]:
unis.shape

(517, 1)

In [239]:
unis.drop_duplicates(keep=False).shape

(0, 1)

In [242]:
unis.reset_index().shape

(517, 3)

In [243]:
unis.reset_index().drop_duplicates(keep=False).shape

(517, 3)